## Обработка данных пользователей социальной сети 'вКонтакте' для заданных групп

### Подгружаем библиотеки

In [30]:
import pandas
import igraph
import sklearn
import urllib
import json
import time
import random

In [2]:
class group_info:
    
    """Класс для сбора информации о пользователях указанной группы 'group_adress'.
    Возвращаются все возможные параметры пользователей, которые можно получить без регистрации приложения.
    Обработка исключений и ошибок пока не реализована"""
    
    def __init__(self, group_adress):
        
        """При создании объекта класса необходимо указать адрес группы вКонтакте"""
        
        self.group_adress = group_adress
        self._url = u'http://api.vk.com/method/groups.getMembers?group_id=' + \
                    group_adress + \
                    '&fields=sex,bdate,city,country,lists,domain,has_mobile,contacts,connections,site,education,universities,schools,status,last_seen,relation,relatives,counters'
        self._url_user = u'http://api.vk.com/method/friends.get?user_id='
        self.count = None
        self.users_info = None
        self.users_friends = None
        
    def get_group_users_count(self):
        
        """Функция возвращает количество пользователей в указанной группе"""
        
        self.count = json.loads(urllib.request.urlopen(self._url).read().decode('utf-8'))['response']['count']
        return self.count
    
    def get_all_group_users_info(self):
        
        """Функция возвращает всю информацию о пользователях указанной группы"""
        
        d = pandas.DataFrame()
        end = int(self.count / 1000)
        for x in range(end + 1):
            resp = json.loads(urllib.request.urlopen(self._url + '&offset=' + str(x * 1000) ).read().decode('utf-8'))
            time.sleep(0.5)
            d = d.append( pandas.DataFrame(resp['response']['users']) )
        self.users_info = d
        return self.users_info
    
    def get_all_group_users_firends(self):
        
        """Функция возвращает всех друзей пользователей группы"""
        
        # total_len = len(uids)
        dd = pandas.DataFrame()
        # for x in range(self.count):
        x = 0
        y = 0
        uids = self.users_info['uid'].tolist()
        while x < self.count:
            if y > 100000:
                break            
            if (x % 10 == 0) and (x != 0):
                time.sleep(1)
            if (x % 100 == 0)  and (x != 0):
                time.sleep(5)
            if (x % 1000 == 0)  and (x != 0):
                print("загружена информация о " + str(x) + " пользователях...")
                print()
                time.sleep(10)
                
            try:
                resp = json.loads(urllib.request.urlopen(self._url_user + str(uids[x])).read().decode('utf-8'))
                if 'error' in resp:
                    if resp['error']['error_code'] == 15:
                        print('Пользователь №' + str(uids[x]) + ' забанен...')
                        x += 1
                        y += 1
                    else:
                        print("загружена информация о " + str(x) + " пользователях...")
                        print('ожидаем...')
                        print('текст ошибки: ')
                        print(resp)
                        print()
                        time.sleep(180)
                        y += 1
                else:
                    # resp_temp = pandas.DataFrame(resp['response'], columns = ['uid2'])#, uid2 = uids.iloc[0])
                    resp_temp = pandas.DataFrame(list(set(resp['response']) & set(uids)), columns = ['uid2'])
                    resp_temp['uid1'] = uids[x]    
                    x += 1
                    y += 1
                time.sleep(0.5)
                dd = dd.append(resp_temp)
            except:
                print("загружена информация о " + str(x) + " пользователях...")
                print('ожидаем...')
                print(resp)
                print()
                y += 1
                time.sleep(180)
                
        self.users_friends = dd
        return self.users_friends

In [3]:
motorchik = group_info('motorchikdeti')

In [4]:
count = motorchik.get_group_users_count()
info = motorchik.get_all_group_users_info()


In [5]:
friend = motorchik.get_all_group_users_firends()

Пользователь №3159771 забанен...
Пользователь №334316191 забанен...
Пользователь №347779089 забанен...
Пользователь №347779473 забанен...


In [11]:
len(friend.values.tolist())

1985

In [21]:
q0 = set(list(friend.ix[:,0]))
q1 = set(list(friend.ix[:,1]))
for x in q1:
    q0.add(x)
vert_dict = dict([[y,x] for x, y in enumerate(q0)])
# print(vert_dict)
#print(friend.values)
#print([(vert_dict[x[0]], vert_dict[x[1]]) for x in friend.values])
g = igraph.Graph(edges = [(vert_dict[v], vert_dict[a]) for v, a in friend.values], directed = True)
#layout = g.layout("fr")
#labels = dict([[x,y] for y, x in zip(users_info['full_name'], users_info['uid'])])

In [28]:
layout = g.layout("fr")
print(g)

IGRAPH D--- 280 1985 --
+ edges:
  0 ->
  1 ->  28 117 128 143 224 253
  2 ->  28
  3 ->  54  68  98 107 112 120 132 147 154 155 177 208 226 232 262
  4 ->  13  15  21  25  28  29  32  33  37  39  41  43  51  52  55  62  73  75
82  84  90  91  98 101 106 115 118 120 131 134 138 139 149 152 156 157 159 173
176 197 202 210 211 220 242 245 246 247 251 252 256 260 263 264 267
  5 ->  19  24  26  28  39  44  50  62  63  67  74  74  81  84  87  91 114 120
124 126 131 158 179 189 200 207 211 223 239 240 247
  6 ->  58 217
  7 ->  24  71  84 120 124 193 207 220 240 277
  8 ->  36  69
  9 -> 120 220 263
 10 ->  28
 11 ->  28  38  83  85 119 165 243 279
 12 ->  58
 13 ->   4  25  33  44  86 120 203 220 242
 14 ->  32 246
 15 ->   4  19  32  40  41  46  52  62 115 120 124 149 176 179 183 184 202 211
219 225 247 278
 16 ->  28 118 192
 17 ->  28 117 143 224
 18 -> 120 131 159 220 256 259
 19 ->   5  15  64  97 172 176 179 219 271
 20 ->  40  55 120
 21 ->   4  28 167 206 219
 22 ->  45  73 233
 23

In [58]:
subgraph_vertex_list = [v.index for v in random.sample(list(g.vs), 100)]
subgraph = igraph.Graph.subgraph(g, subgraph_vertex_list)
for x in g.vs.indices:
    print(x)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [61]:
for x in zip(g.vs.indices, igraph.Graph.pagerank(g)):
    print(x)

(0, 0.0010019314390566705)
(1, 0.0034706857539136948)
(2, 0.0009637657698467073)
(3, 0.004920182931108896)
(4, 0.02605910138103318)
(5, 0.011709255915421636)
(6, 0.001528840352539107)
(7, 0.0038191299233831773)
(8, 0.0015679507018165743)
(9, 0.0016619453984701976)
(10, 0.0009637657698467073)
(11, 0.0038539931117113025)
(12, 0.0009693324693706609)
(13, 0.0038681045748201253)
(14, 0.001415931047070471)
(15, 0.009512530528787075)
(16, 0.0024582408483103204)
(17, 0.0024558196830325517)
(18, 0.00309842756043136)
(19, 0.004520792797381534)
(20, 0.0016729431023817165)
(21, 0.0026790637379339715)
(22, 0.001790084793547104)
(23, 0.001695588311950231)
(24, 0.009944915033396153)
(25, 0.023218638016185237)
(26, 0.006055261685161825)
(27, 0.002822482790604707)
(28, 0.036754599301701024)
(29, 0.0009443161284492758)
(30, 0.001000557638715604)
(31, 0.001494525213184107)
(32, 0.007154878724569167)
(33, 0.003979941405517723)
(34, 0.0013817975494073767)
(35, 0.0017299410266151914)
(36, 0.0020891178951811

In [39]:
#igraph.plot(subgraph.vs)
            #"D:/social_network.png",
            #edge_curved = True
           # layout = layout
           #)

C:\Anaconda3\lib\site-packages\igraph\drawing\__init__.py:263: UserWarning: <igraph.VertexSeq object at 0x000000A6420F79A8> does not support plotting
  warn("%s does not support plotting" % obj)


AttributeError: 'bytes' object has no attribute 'encode'

In [ ]:
class all_groups_info:
    
    """Класс для работы сразу с несколькими группами"""
    
    def __init__(self, groups_list):
        
        """При создании объекта наобходимо передать список групп, которые необходимо исследовать"""
        
        self.groups_list = groups_list
        
    def get_all_groups_users_info:
        pass
        
    
    

In [75]:
citilink_group.count

71804

In [76]:
citilink_group = group_info("citilink_ru")
count = citilink_group.get_group_users_count()
print(count)

71804


In [77]:
info = citilink_group.get_all_group_users_info()

0            858
1           2111
2           2345
3           2962
4           3411
5           4143
6           4435
7           5056
8           5294
9           5310
10          5432
11          6172
12          6669
13          6982
14          7043
15          7834
16          8288
17          9106
18          9435
19         10197
20         10926
21         11646
22         12553
23         12714
24         12880
25         13327
26         13907
27         14155
28         14181
29         14682
         ...    
775    367231493
776    367233221
777    367299342
778    367302609
779    367391327
780    367453160
781    367492145
782    367600343
783    367717842
784    367774707
785    367826574
786    367832019
787    368092484
788    368131523
789    368238266
790    368240647
791    368248332
792    368293698
793    368355147
794    368489920
795    368558875
796    369019702
797    369083611
798    369257537
799    369735304
800    369766559
801    370011388
802    3700852

### Подгружаем информацию о пользователях группы

In [44]:
# задаем список групп и нужные URL

groups_list = ['citilink_ru', 'i_love_ulmart']
url = u'http://api.vk.com/method/groups.getMembers?group_id=citilink_ru&fields=sex,bdate,city,country,lists,domain,has_mobile,contacts,connections,site,education,universities,schools,status,last_seen,relation,relatives,counters'
url2 = u'http://api.vk.com/method/friends.get?user_id='

In [36]:
# получаем количество человек в группе

group_count_url = urllib.request.urlopen(url).read().decode('utf-8')
group_count = json.loads(group_count_url)
total_users = group_count['response']['count']
# print(pandas.DataFrame(group_count['response']['users']))
print(total_users)

71789


In [37]:
def get_all_users(query, gc):
    
    """
    Function gets all VK group users from required URL
    using offset to get one by one by 1000 users
    """
    
    d = pandas.DataFrame()
    end = int(gc / 1000)
    for x in range(end + 1):
        resp = json.loads(urllib.request.urlopen(query + '&offset=' + str(x * 1000) ).read().decode('utf-8'))
        # print(x)
        # print(resp)
        time.sleep(0.34)
        d = d.append( pandas.DataFrame(resp['response']['users']) )
    return d

In [38]:
vk_group_users = get_all_users(url, total_users)

In [39]:
print(vk_group_users)

          bdate   city  country deactivated               domain  \
0          16.8    2.0      1.0         NaN              denisko   
1      4.3.1986    1.0      1.0         NaN               id2111   
2    30.10.1986    2.0      1.0         NaN     nadejda_sergeeva   
3          14.6    2.0      1.0         NaN               id2962   
4    12.11.1988    2.0      1.0         NaN               id3411   
5      3.8.1988    2.0      1.0         NaN  konstantin.kiryanov   
6           NaN    2.0      1.0         NaN               id4435   
7           NaN    2.0      1.0         NaN     volodchenko_anna   
8    10.10.1986    1.0      1.0         NaN              shinkov   
9     12.4.1989    2.0      1.0         NaN               id5310   
10         19.3  109.0      1.0         NaN                melza   
11         22.3    1.0      1.0         NaN               uvarov   
12   31.12.1983    1.0      1.0         NaN               id6669   
13    24.2.1982   40.0      1.0         NaN     

In [45]:
# Получаем список всех пользователей группы

uids = vk_group_users['uid'].tolist()
print(uids[2])

2345


In [46]:
# Пример получения списка друзей для заданного пользователя
print(uids[0])
resp = json.loads(urllib.request.urlopen(url2 + str(uids[1])).read().decode('utf-8'))
print(list(set(resp['response']) & set(uids)))
resp_temp = pandas.DataFrame(list(set(resp['response']) & set(uids)), columns = ['uid2'])#, uid2 = uids.iloc[0])
resp_temp['uid1'] = uids[0]
print(resp_temp)

858
[26739296, 471585, 93206849, 2815203, 1276708, 20603205, 1310373, 3413702, 419553, 191270793, 2606666, 164873324, 4626863, 8505297, 175568787, 1202110, 1297173, 11069946, 94475996, 5127230]
         uid2  uid1
0    26739296   858
1      471585   858
2    93206849   858
3     2815203   858
4     1276708   858
5    20603205   858
6     1310373   858
7     3413702   858
8      419553   858
9   191270793   858
10    2606666   858
11  164873324   858
12    4626863   858
13    8505297   858
14  175568787   858
15    1202110   858
16    1297173   858
17   11069946   858
18   94475996   858
19    5127230   858


In [ ]:
def get_all_users_friends(uids):
    total_len = len(uids)
    dd = pandas.DataFrame()
    for x in range(total_len):

        if x % 10 == 0:
            time.sleep(1)
        if x % 100 == 0:
            time.sleep(5)
        if x % 1000 == 0:
            time.sleep(10)
            print x
        try:
            resp = json.loads(urllib2.urlopen(url2 + str(uids[x])).read())
            if 'error' in resp:
                pass
            else:
                # resp_temp = pandas.DataFrame(resp['response'], columns = ['uid2'])#, uid2 = uids.iloc[0])
                resp_temp = pandas.DataFrame(list(set(resp['response']) & set(uids)), columns = ['uid2'])
                resp_temp['uid1'] = uids[x]    
            time.sleep(0.5)
            dd = dd.append(resp_temp)
        except:
            print('Waiting...')
            print(json.loads(r.text))
            time.sleep(20)

    return dd

In [ ]:
for user in userIDs:
    # используем try-except, чтобы не брать удаленных пользователей и не попасть в бан
    try:
        r = requests.get('https://api.vk.com/method/friends.get?user_id=' + str(user))

        info = json.loads(r.text)['response']

        idsList = []
        for uid in info:
            idsList.append(uid)

        userInfo['id'] = user;
        userInfo['friends'] = idsList

        with open("users.txt", "a") as f:
            f.write(str(userInfo) + '\n')

        if j%100==0:
            print j

        j += 1
        time.sleep(0.5)
    except:
        print 'Waiting...'
        print json.loads(r.text)
        time.sleep(20)

In [122]:
total_len = len(uids)
# dd = pandas.DataFrame()
for x in range(52000, total_len):#range(total_len):

    if x % 10 == 0:
        time.sleep(3)
    if x % 100 == 0:
        time.sleep(60)
    if x % 1000 == 0:
        time.sleep(240)
        print x
    
    resp = json.loads(urllib2.urlopen(url2 + str(uids[x])).read())
    if 'error' in resp:
        pass
    else:
        # resp_temp = pandas.DataFrame(resp['response'], columns = ['uid2'])#, uid2 = uids.iloc[0])
        resp_temp = pandas.DataFrame(list(set(resp['response']) & set(uids)), columns = ['uid2'])
        resp_temp['uid1'] = uids[x]    
    time.sleep(0.34)
    dd = dd.append(resp_temp)
print dd

52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
         uid2       uid1
0     5083227        953
1       20767        953
0    26739296       2111
1      471585       2111
2     2815203       2111
3     1276708       2111
4     1310373       2111
5     3413702       2111
6      419553       2111
7     2606666       2111
8   133664843       2111
9    93206849       2111
10    4626863       2111
11    8505297       2111
12  175568787       2111
13    1297173       2111
14    1202110       2111
15   11069946       2111
16   94475996       2111
17    5127230       2111
18    1427071       2111
0    26739296       2111
1      471585       2111
2     2815203       2111
3     1276708       2111
4     1310373       2111
5     3413702       2111
6      419553       2111
7     2606666       2111
8   133664843       2111
..        ...        ...
0    78988006  343475094
0   260336939  343587776
0   188186400  343599305
0    74906646  34362768

In [123]:
dd.to_csv('D://citi_group_all.csv')
print(resp)

{u'response': []}


In [126]:
vk_group_users.to_csv(u'D://vk_group_users.csv', encoding='utf8')